## Airport Simulation

Here this simulation is a 2 Process Simulation where we enter a process A(Check In) and the when that is complete we enter the 2nd Process B(Immigration). The CheckIn is a random distribution from value 3 to 7 min and Immigration is a normal distribution with a mean of 7 min and std dev with 2 mins.

In [1]:
from random import randrange
import datetime

In [2]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + datetime.timedelta(seconds=random_second)

In [3]:
d1 = datetime.datetime.strptime('1/1/2009 9:00 AM', '%m/%d/%Y %I:%M %p')
d2 = datetime.datetime.strptime('1/1/2009 6:00 PM', '%m/%d/%Y %I:%M %p')


N = 50
arrival_times = []
for i in range(N):
    arrival_times.append(random_date(d1, d2))
    
arrival_times[:5]

[datetime.datetime(2009, 1, 1, 16, 36, 37),
 datetime.datetime(2009, 1, 1, 14, 1, 57),
 datetime.datetime(2009, 1, 1, 13, 17, 13),
 datetime.datetime(2009, 1, 1, 12, 2, 13),
 datetime.datetime(2009, 1, 1, 15, 9, 46)]

In [4]:
import pandas as pd
import numpy as np


df = pd.DataFrame()
df['Passengers'] = list(np.arange(N) + 1)
df['Airport Arrival Time'] = arrival_times
df['Airport Arrival Time'] = df['Airport Arrival Time'].sort_values(ascending=True).tolist()

df

,Passengers,Airport Arrival Time
0,1,2009-01-01 09:07:37
1,2,2009-01-01 09:41:45
2,3,2009-01-01 09:49:30
3,4,2009-01-01 09:54:21
4,5,2009-01-01 09:57:58
5,6,2009-01-01 10:07:55
6,7,2009-01-01 10:10:40
7,8,2009-01-01 10:42:44
8,9,2009-01-01 10:49:05
9,10,2009-01-01 11:12:02


In [5]:
df.dtypes

Passengers                       int64
Airport Arrival Time    datetime64[ns]
dtype: object

In [6]:
def time2timedelta(timeObj):
    return datetime.datetime.combine(datetime.date.min, timeObj) - datetime.datetime.min

def timedelta2time(deltaObj):
    return (datetime.datetime.min + deltaObj).time()


In [7]:
import math

def checkIn_process(low_val, high_val):
    random_number = np.random.uniform(low=low_val, high=high_val)
    hours = math.floor(random_number/60)
    minutes = math.floor(random_number) if random_number < 60 else (math.floor(random_number) - (hours*60))
    seconds = round(((random_number - math.floor(random_number))*0.6)*100)
    x = timedelta2time(datetime.timedelta(hours = hours, minutes = minutes, seconds = seconds))
    return x 

low_val = 3.0
high_val = 7.0


def immigration_Process(mu, sigma):
    random_number = np.random.normal(mu, sigma)
    hours = math.floor(random_number/60)
    minutes = math.floor(random_number) if random_number < 60 else (math.floor(random_number) - (hours*60))
    seconds = round(((random_number - math.floor(random_number))*0.6)*100)
    x = timedelta2time(datetime.timedelta(hours = hours, minutes = minutes, seconds = seconds))
    return x 

mu = 8
sigma = 2

In [8]:
checkin_queue, checkin_start, checkin_process, checkin_finish = [], [], [], []
immigration_queue, immigration_start, immigration_process, immigration_finish = [], [], [], []
total_time = []

In [9]:
checkin_queue.append(timedelta2time(datetime.timedelta(seconds=0)))
checkin_start.append(df['Airport Arrival Time'][0].to_pydatetime())

checkinProcess = checkIn_process(low_val, high_val)
checkin_process.append(checkinProcess)

checkinFinish = checkin_start[0] + time2timedelta(checkin_process[0]) 
checkin_finish.append(checkinFinish)


immigration_queue.append(timedelta2time(datetime.timedelta(seconds=0)))
immigration_start.append(checkin_finish[0])

immigrationProcess = immigration_Process(mu, sigma)
immigration_process.append(immigrationProcess)

immigrationFinish = immigration_start[0] + time2timedelta(immigration_process[0]) 
immigration_finish.append(immigrationFinish)

total_time.append(timedelta2time(immigration_finish[-1] - checkin_start[-1]))

In [10]:
for index, row in df.iterrows():
    if index == 0:
        continue
    checkin_arrivalTime = df['Airport Arrival Time'][index].to_pydatetime()
    if checkin_arrivalTime >=  checkin_finish[-1]:
        checkin_start.append(checkin_arrivalTime)
        checkin_queue.append(timedelta2time(datetime.timedelta(seconds=0)))
        
        checkinProcess = checkIn_process(low_val, high_val)
        checkin_process.append(checkinProcess)
        
        checkinFinish = checkin_start[-1] + time2timedelta(checkin_process[-1]) 
        checkin_finish.append(checkinFinish)
    else:
        checkin_start.append(checkin_finish[-1])
        checkin_queue.append(timedelta2time(checkin_finish[-1] - checkin_arrivalTime))
        
        checkinProcess = checkIn_process(low_val, high_val)
        checkin_process.append(checkinProcess)
        
        checkinFinish = checkin_start[-1] + time2timedelta(checkin_process[-1]) 
        checkin_finish.append(checkinFinish)
    
    if checkin_finish[-1] >= immigration_finish[-1]:
        immigration_start.append(checkin_finish[-1])
        immigration_queue.append(timedelta2time(datetime.timedelta(seconds=0)))

        immigrationProcess = immigration_Process(mu, sigma)
        immigration_process.append(immigrationProcess)

        immigrationFinish = immigration_start[-1] + time2timedelta(immigration_process[-1]) 
        immigration_finish.append(immigrationFinish)
    else:
        immigration_start.append(immigration_finish[-1])
        immigration_queue.append(timedelta2time(immigration_finish[-1] - checkin_finish[-1]))

        immigrationProcess = immigration_Process(mu, sigma)
        immigration_process.append(immigrationProcess)

        immigrationFinish = immigration_start[-1] + time2timedelta(immigration_process[-1]) 
        immigration_finish.append(immigrationFinish)
    total_time.append(timedelta2time(immigration_finish[-1] - checkin_arrivalTime))

In [11]:
df['Check-In Queue Time'] = checkin_queue
df['Check-In Start Time'] = checkin_start
df['Check-In Process Time'] = checkin_process
df['Check-In Finish Time'] = checkin_finish

df['Immigration Queue Time'] = immigration_queue
df['Immigration Start Time'] = immigration_start
df['Immigration Process Time'] = immigration_process
df['Immigration Finish Time'] = immigration_finish

df['Total Time Spent'] = total_time

df      

,Passengers,Airport Arrival Time,Check-In Queue Time,Check-In Start Time,Check-In Process Time,Check-In Finish Time,Immigration Queue Time,Immigration Start Time,Immigration Process Time,Immigration Finish Time,Total Time Spent
0,1,2009-01-01 09:07:37,00:00:00,2009-01-01 09:07:37,00:06:25,2009-01-01 09:14:02,00:00:00,2009-01-01 09:14:02,00:07:40,2009-01-01 09:21:42,00:14:05
1,2,2009-01-01 09:41:45,00:00:00,2009-01-01 09:41:45,00:03:39,2009-01-01 09:45:24,00:00:00,2009-01-01 09:45:24,00:07:03,2009-01-01 09:52:27,00:10:42
2,3,2009-01-01 09:49:30,00:00:00,2009-01-01 09:49:30,00:04:58,2009-01-01 09:54:28,00:00:00,2009-01-01 09:54:28,00:07:47,2009-01-01 10:02:15,00:12:45
3,4,2009-01-01 09:54:21,00:00:07,2009-01-01 09:54:28,00:05:22,2009-01-01 09:59:50,00:02:25,2009-01-01 10:02:15,00:07:27,2009-01-01 10:09:42,00:15:21
4,5,2009-01-01 09:57:58,00:01:52,2009-01-01 09:59:50,00:03:07,2009-01-01 10:02:57,00:06:45,2009-01-01 10:09:42,00:10:25,2009-01-01 10:20:07,00:22:09
5,6,2009-01-01 10:07:55,00:00:00,2009-01-01 10:07:55,00:05:17,2009-01-01 10:13:12,00:06:55,2009-01-01 10:20:07,00:09:15,2009-01-01 10:29:22,00:21:27
6,7,2009-01-01 10:10:40,00:02:32,2009-01-01 10:13:12,00:06:30,2009-01-01 10:19:42,00:09:40,2009-01-01 10:29:22,00:09:23,2009-01-01 10:38:45,00:28:05
7,8,2009-01-01 10:42:44,00:00:00,2009-01-01 10:42:44,00:03:43,2009-01-01 10:46:27,00:00:00,2009-01-01 10:46:27,00:06:30,2009-01-01 10:52:57,00:10:13
8,9,2009-01-01 10:49:05,00:00:00,2009-01-01 10:49:05,00:05:14,2009-01-01 10:54:19,00:00:00,2009-01-01 10:54:19,00:11:47,2009-01-01 11:06:06,00:17:01
9,10,2009-01-01 11:12:02,00:00:00,2009-01-01 11:12:02,00:04:17,2009-01-01 11:16:19,00:00:00,2009-01-01 11:16:19,00:07:27,2009-01-01 11:23:46,00:11:44
